In [0]:
%fs ls /FileStore/tables/textcorpora

path,name,size
dbfs:/FileStore/tables/textcorpora/austen_emma.txt,austen_emma.txt,903894
dbfs:/FileStore/tables/textcorpora/bible_kjv.txt,bible_kjv.txt,4432359
dbfs:/FileStore/tables/textcorpora/blake_poems.txt,blake_poems.txt,39594
dbfs:/FileStore/tables/textcorpora/burgess_busterbrown.txt,burgess_busterbrown.txt,86334
dbfs:/FileStore/tables/textcorpora/carroll_alice.txt,carroll_alice.txt,147726
dbfs:/FileStore/tables/textcorpora/chesterton_brown.txt,chesterton_brown.txt,414283
dbfs:/FileStore/tables/textcorpora/melville_moby_dick-1.txt,melville_moby_dick-1.txt,1265914
dbfs:/FileStore/tables/textcorpora/melville_moby_dick.txt,melville_moby_dick.txt,1265914
dbfs:/FileStore/tables/textcorpora/milton_paradise-1.txt,milton_paradise-1.txt,478855
dbfs:/FileStore/tables/textcorpora/milton_paradise.txt,milton_paradise.txt,478855


In [0]:
f = sc.wholeTextFiles('/FileStore/tables/textcorpora')

In [0]:
# f.take(1)

In [0]:
import re
def filename(path):
  return re.match('dbfs:/FileStore/tables/textcorpora/(.+)\.txt', path)[1]

In [0]:
f2 = f.map(lambda t: (filename(t[0]), t[1]))

In [0]:
f3 = f2.map(lambda t: (t[0], t[1].split()))

In [0]:
def tokenPairs(docid, tokens):
  return [(docid, token) for token in tokens]

In [0]:
f4 = f3.flatMap(lambda t: tokenPairs(t[0], t[1]))

In [0]:
f4.take(10)

Out[126]: [('austen_emma', '[Emma'),
 ('austen_emma', 'by'),
 ('austen_emma', 'Jane'),
 ('austen_emma', 'Austen'),
 ('austen_emma', '1816]'),
 ('austen_emma', 'VOLUME'),
 ('austen_emma', 'I'),
 ('austen_emma', 'CHAPTER'),
 ('austen_emma', 'I'),
 ('austen_emma', 'Emma')]

In [0]:
f5 = f4.map(lambda t: (t[0], t[1].lower()))

In [0]:
f5.take(5)

Out[128]: [('austen_emma', '[emma'),
 ('austen_emma', 'by'),
 ('austen_emma', 'jane'),
 ('austen_emma', 'austen'),
 ('austen_emma', '1816]')]

In [0]:
f6 = f5.map(lambda t: (t[0], "".join(filter(lambda c: 97 <= ord(c) <= 122, t[1]))))

In [0]:
f6.take(5)

Out[130]: [('austen_emma', 'emma'),
 ('austen_emma', 'by'),
 ('austen_emma', 'jane'),
 ('austen_emma', 'austen'),
 ('austen_emma', '')]

In [0]:
# f7 -- ('austen_emma', 'emma')
f7 = f6.filter(lambda t: len(t[1]) > 0)
f7.cache()

Out[131]: PythonRDD[601] at RDD at PythonRDD.scala:58

In [0]:
f7.take(5)

Out[132]: [('austen_emma', 'emma'),
 ('austen_emma', 'by'),
 ('austen_emma', 'jane'),
 ('austen_emma', 'austen'),
 ('austen_emma', 'volume')]

In [0]:
f7.count()

Out[133]: 1935262

In [0]:
STOPWORDS = frozenset({
  "a", "as", "able", "about", "above", "according", "accordingly",
  "across", "actually", "after", "afterwards", "again", "against", "aint", "all", "allow",
  "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among",
  "amongst", "an", "and", "another", "any", "anybody", "anyhow", "anyone", "anything", "anyway",
  "anyways", "anywhere", "apart", "appear","appreciate", "appropriate", "are", "arent", "around",
  "as", "aside", "ask", "asking", "associated", "at", "available", "away", "awfully", "be", "became",
  "because", "become", "becomes", "becoming", "been", "before", "beforehand", "behind", "being", "believe", "below",
  "beside", "besides", "best", "better", "between", "beyond", "both", "brief", "but", "by", "cmon",
  "cs", "came", "can", "cant", "cannot", "cant", "cause", "causes", "certain", "certainly", "changes",
  "clearly", "co", "com", "come", "comes", "concerning", "consequently", "consider", "considering", "contain", "containing",
  "contains", "corresponding", "could", "couldnt", "course", "currently", "definitely", "described", "despite", "did", "didnt",
  "different", "do", "does", "doesnt", "doing", "dont", "done", "down", "downwards", "during", "each",
  "edu", "eg", "eight", "either", "else", "elsewhere", "enough", "entirely", "especially", "et", "etc",
  "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except",
  "far", "few", "ff", "fifth", "first", "five", "followed", "following", "follows", "for", "former",
  "formerly", "forth", "four", "from", "further", "furthermore", "get", "gets", "getting", "given", "gives",
  "go", "goes", "going", "gone", "got", "gotten", "greetings", "had", "hadnt", "happens", "hardly",
  "has", "hasnt", "have", "havent", "having", "he", "hes", "hello", "help", "hence", "her",
  "here", "heres", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "hi", "him", "himself",
  "his", "hither", "hopefully", "how", "howbeit", "however", "i", "id", "ill", "im", "ive",
  "ie", "if", "ignored", "immediate", "in", "inasmuch", "inc", "indeed", "indicate", "indicated", "indicates",
  "inner", "insofar", "instead", "into", "inward", "is", "isnt", "it", "itd", "itll", "its",
  "its", "itself", "just", "keep", "keeps", "kept", "know", "knows", "known", "last", "lately",
  "later", "latter", "latterly", "least", "less", "lest", "let", "lets", "like", "liked", "likely",
  "little", "look", "looking", "looks", "ltd", "mainly", "many", "may", "maybe", "me", "mean",
  "meanwhile", "merely", "might", "more", "moreover", "most", "mostly", "much", "must", "my", "myself",
  "name", "namely", "nd", "near", "nearly", "necessary", "need", "needs", "neither", "never", "nevertheless",
  "new", "next", "nine", "no", "nobody", "non", "none", "noone", "nor", "normally", "not",
  "nothing", "novel", "now", "nowhere", "obviously", "of", "off", "often", "oh", "ok", "okay",
  "old", "on", "once", "one", "ones", "only", "onto", "or", "other", "others", "otherwise",
  "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "own", "particular", "particularly",
  "per", "perhaps", "placed", "please", "plus", "possible", "presumably", "probably", "provides", "que", "quite",
  "qv", "rather", "rd", "re", "really", "reasonably", "regarding", "regardless", "regards", "relatively", "respectively",
  "right", "said", "same", "saw", "say", "saying", "says", "second", "secondly", "see", "seeing",
  "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously",
  "seven", "several", "shall", "she", "should", "shouldnt", "since", "six", "so", "some", "somebody",
  "somehow", "someone", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "specified", "specify",
  "specifying", "still", "sub", "such", "sup", "sure", "ts", "take", "taken", "tell", "tends",
  "th", "than", "thank", "thanks", "thanx", "that", "thats", "thats", "the", "their", "theirs",
  "them", "themselves", "then", "thence", "there", "theres", "thereafter", "thereby", "therefore", "therein", "theres",
  "thereupon", "these", "they", "theyd", "theyll", "theyre", "theyve", "think", "third", "this", "thorough",
  "thoroughly", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too",
  "took", "toward", "towards", "tried", "tries", "truly", "try", "trying", "twice", "two", "un",
  "under", "unfortunately", "unless", "unlikely", "until", "unto", "up", "upon", "us", "use", "used",
  "useful", "uses", "using", "usually", "value", "various", "very", "via", "viz", "vs", "want",
  "wants", "was", "wasnt", "way", "we", "wed", "well", "were", "weve", "welcome", "well",
  "went", "were", "werent", "what", "whats", "whatever", "when", "whence", "whenever", "where", "wheres",
  "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who",
  "whos", "whoever", "whole", "whom", "whose", "why", "will", "willing", "wish", "with", "within",
  "without", "wont", "wonder", "would", "would", "wouldnt", "yes", "yet", "you", "youd", "youll",
  "youre", "youve", "your", "yours", "yourself", "yourselves", "zero"})

In [0]:
doc_terms = f7.filter(lambda record: record[1] not in STOPWORDS).toDF(schema=['doc_id', 'term'])
doc_terms.printSchema()
doc_terms.take(5)

root
-- doc_id: string (nullable = true)
-- term: string (nullable = true)

Out[135]: [Row(doc_id='austen_emma', term='emma'),
 Row(doc_id='austen_emma', term='jane'),
 Row(doc_id='austen_emma', term='austen'),
 Row(doc_id='austen_emma', term='volume'),
 Row(doc_id='austen_emma', term='chapter')]

In [0]:
doc_term_count = doc_terms.groupBy("doc_id", "term").count()
doc_term_count.printSchema()

root
-- doc_id: string (nullable = true)
-- term: string (nullable = true)
-- count: long (nullable = false)

In [0]:
doc_size = doc_term_count.groupBy("doc_id").sum("count").withColumnRenamed("sum(count)", "doc_size")

In [0]:
term_docs = doc_term_count.groupBy("term").count().withColumnRenamed("count", "term_ndocs")

In [0]:
from pyspark.sql.functions import col

tfidf = doc_term_count \
  .join(doc_size, on="doc_id") \
  .join(term_docs, on="term") \
  .withColumn("tfidf", 1000000 * (col("count") / col("doc_size")) / col("term_ndocs")) \
  .select(col("doc_id"), col("term"), col("tfidf"))

tfidf.printSchema()

root
-- doc_id: string (nullable = true)
-- term: string (nullable = true)
-- tfidf: double (nullable = true)

In [0]:
tfidf.sort(col("tfidf").desc()).take(5)

Out[140]: [Row(doc_id='carroll_alice', term='alice', tfidf=45513.65409622886),
 Row(doc_id='burgess_busterbrown', term='buster', tfidf=40585.00914076783),
 Row(doc_id='austen_emma', term='emma', tfidf=14027.943813508668),
 Row(doc_id='burgess_busterbrown', term='blacky', tfidf=9323.583180987203),
 Row(doc_id='burgess_busterbrown', term='browns', tfidf=8866.54478976234)]

In [0]:
import re

_RE_WORDS = re.compile(r"[a-z]+")

def relevance(query, df, top_n=5):
  terms = [term for term in _RE_WORDS.findall(query.lower()) if term not in STOPWORDS]
  df_query = sc.parallelize([[term] for term in terms]).toDF(schema=['term'])
  return df \
    .join(df_query, on="term") \
    .groupBy("doc_id").sum("tfidf") \
    .withColumn("rank", col("sum(tfidf)") / len(terms)) \
    .select(col("doc_id"), col("rank")) \
    .sort(col("rank").desc()) \
    .take(top_n)

def pprint_df(data):
  for (i, row) in enumerate(data, 1):
    print("%d. %-20s %.4f" % (i, row.doc_id, row.rank))

In [0]:
pprint_df(relevance('dead KING garden!', tfidf, top_n=5))

1. carroll_alice 249.8979
2. bible_kjv 230.7691
3. chesterton_brown 100.9568
4. blake_poems 86.5316
5. milton_paradise-1 54.6957

In [0]:
pprint_df(relevance('Into what dangers, would you leade me Cassius?', tfidf, top_n=5))

1. shakespeare_caesar-1 1635.2695
2. shakespeare_caesar 1635.2695
3. milton_paradise-1 2.4517
4. milton_paradise 2.4517
5. austen_emma 1.7790